In [ ]:
import dalex as dx
import pandas as pd
import pickle
import numpy as np

from sklearn.model_selection import train_test_split

In [ ]:
input_df = pd.read_csv('preprocessed_dataset.csv')
y = input_df.loc[:,'Attrition']
X = input_df.drop('Attrition', axis='columns')

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=14)

In [ ]:
lr = pickle.load(open( 'l1_log_reg.p', "rb" ))
lr_explainer = dx.Explainer(lr, X_train, y_train, label='Logistic Regresion')

In [ ]:
XGB = pickle.load(open( 'xgb_model.p', "rb" ))
XGB_explainer = dx.Explainer(XGB, X_train, y_train, label='XGBoost')

In [ ]:
RF = pickle.load(open( 'random_forest_model.p', "rb" ))
RF_explainer = dx.Explainer(RF, X_train, y_train, label='Random Forest')